#### Managing agent steps
In this example, we will build a chat executor that better manages intermediate steps. The base chat executor just puts all the messages into the model, but if the intermediate steps taken by the agent are getting too long, we would need to modify that. In this example, we would only be providing the 10 most recent messages to the model

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=3)]

In [3]:
from langgraph.prebuilt import ToolExecutor
tool_executor = ToolExecutor(tools)

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="mistralai/Mixtral-8x7B-Instruct-v0.1")

In [5]:
from langchain.tools.render import format_tool_to_openai_function

functions = [format_tool_to_openai_function(tool) for tool in tools]
model = model.bind_functions(functions)

In [6]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [7]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage


def should_continue(state: AgentState):
    messages = state["messages"]
    last_message: BaseMessage = messages[-1]
    
    if "tool_calls" in last_message.additional_kwargs:
        return "continue"
    else:
        return "end"

In [8]:
def call_model(state: AgentState):
    messages = state["messages"][-5:]
    response = model.invoke(messages)
    return {"messages": [response]}

def call_tool(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    fn = last_message.additional_kwargs["tool_calls"][0]["function"]
    action = ToolInvocation(
        tool=fn["name"], 
        tool_input=json.loads(fn["arguments"])
    )
    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=action.tool)
    return {"messages": [function_message]}

In [11]:
from langgraph.graph import StateGraph, END 

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)



workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue, 
    {
        "continue": "action",
        "end": END
    }
)

workflow.add_edge("action", "agent")


app = workflow.compile()

In [13]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="who is the current president of Nigeria?")]}

for output in app.stream(inputs):
    for key, value in output.items():
        print(f"output from node {key}")
        print("----")
        print(f"{value}")
    print("----")

output from node agent
----
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0o53ewxjjaqo11jz73esatpm', 'function': {'arguments': '{"query":"current president of Nigeria"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]})]}
----
output from node action
----
{'messages': [FunctionMessage(content="[{'url': 'https://www.aljazeera.com/news/2023/3/1/who-is-nigerias-new-president-elect-bola-tinubu', 'content': 'Tinubu never hid his ambition to be president of Nigeria as the 70-year-old fulfilled his “lifelong” goal by winning the presidency of Africa’s most populous nation with 8.8 million votes, according to final election results.\\n However, some critics say the pro-Tinubu narrative erases the welfarist politics of Lateef Jakande, governor between 1979 and 1983, who laid the foundations of today’s Lagos, which has swelled from four million people three decades ago to 21 million today.\\n Often referred to by the chieftain’s title of “Jagaban